In [3]:
import pandas as pd
import numpy as np
import pyodbc
import json
#from google.cloud import bigquery
from pandas.io import gbq
from datetime import datetime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table, create_engine
from sqlalchemy.orm import sessionmaker


from statsmodels.stats.weightstats import ztest as ztest
from statsmodels.stats.stattools import durbin_watson
import sqlalchemy

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import matplotlib.pyplot as plt
%matplotlib inline
import random
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error


import seaborn as sns
import plotly.express as px

import scipy 
from scipy.stats import iqr
from scipy.stats import norm
from scipy.stats import t
from scipy import stats

import math

In [45]:
moved_cats = pd.read_excel('/home/a.kosheleva/moved_sweet_cats_2.xlsx', engine= 'openpyxl')

In [46]:
moved_cats.head()

,date,value,period,category
0,2022-10-01,6.612336e+06,pre_pilot,sweets
1,2022-10-02,6.071246e+06,pre_pilot,sweets
2,2022-10-03,5.950962e+06,pre_pilot,sweets
3,2022-10-04,6.274467e+06,pre_pilot,sweets
4,2022-10-05,5.813274e+06,pre_pilot,sweets


In [47]:
df_prepilot = moved_cats[moved_cats['period'] == 'pre_pilot']
df_pilot = moved_cats[moved_cats['period'] == 'pilot']

In [48]:
df_pilot

,date,value,period,category
37,2022-11-07,5.480441e+06,pilot,sweets
38,2022-11-08,5.052774e+06,pilot,sweets
39,2022-11-09,5.531690e+06,pilot,sweets
40,2022-11-10,5.947663e+06,pilot,sweets
41,2022-11-11,7.245025e+06,pilot,sweets
42,2022-11-12,6.923377e+06,pilot,sweets
43,2022-11-13,5.748137e+06,pilot,sweets
81,2022-11-07,4.919751e+06,pilot,other_cats
82,2022-11-08,4.693106e+06,pilot,other_cats
83,2022-11-09,4.784316e+06,pilot,other_cats


In [49]:
l_test = ['sweets']
l_contrl = ['other_cats']

In [50]:
def forecast_for_ab(df_prepilot, df_pilot, mtr = 'value', corr_lim = 0, control_share = 0.1, sigma = 1.5):
        y_train = df_prepilot.loc[
        df_prepilot['category'].isin(l_test)
        ].pivot_table(index='date', aggfunc={mtr:'sum'})[[mtr]].values
        X_train = df_prepilot.loc[
            df_prepilot['category'].isin(l_contrl)
        ].pivot_table(index='date', aggfunc={mtr:'sum'}).reset_index()[[mtr]].values
        X_train = sm.add_constant(X_train)
        y_test = df_pilot.loc[
            df_pilot['category'].isin(l_test)
        ].pivot_table(index='date', aggfunc={mtr:'sum'})[[mtr]].values
        X_test = df_pilot.loc[
            df_pilot['category'].isin(l_contrl)
        ].pivot_table(index='date', aggfunc={mtr:'sum'}).reset_index()[[mtr]].values
        X_test = sm.add_constant(X_test)

        # обучаем модель
        model = sm.OLS(y_train, X_train)
        results = model.fit()

        # оцениваем точность модели
        float_rsquared = round(results.rsquared, 4)
        series_predictions = pd.Series(results.predict(X_test))
        series_facts = pd.Series([i[0] for i in y_test])
        series_prepilot_predictions = pd.Series(results.predict())
        series_prepilot_facts = pd.Series([i[0] for i in y_train])
        rmse = mean_squared_error(series_predictions, series_facts)**(1/2)
        rmse_prec = round(rmse / series_predictions.mean(), 4)
        print(f'R-squared: {float_rsquared}, holdout {sigma} rmse (%): {sigma * rmse_prec}')
        print(f'Control group size: {len(l_contrl)}, Test group size: {len(l_test)}')
        # визуализация точности прогнозирования
        t_prepilot = [i + 1 for i in range(len(series_prepilot_predictions))]
        t_pilot = [i + 1 for i in range(max(t_prepilot), max(t_prepilot) + len(series_predictions))]
        df_pilot_pred = pd.DataFrame({
            'data_type':'forecast',
            'period_type':'pilot',
            'values':series_predictions,
            'ci_low':series_predictions - rmse * sigma,
            'ci_up':series_predictions + rmse * sigma,
            't':t_pilot
        })
        df_pilot_fact = pd.DataFrame({
            'data_type':'fact',
            'period_type':'pilot',
            'values':series_facts,
            'ci_low':series_predictions - rmse * sigma,
            'ci_up':series_predictions + rmse * sigma,
            't':t_pilot
        })
        df_prepilot_pred = pd.DataFrame({
            'data_type':'forecast',
            'period_type':'prepilot',
            'values':series_prepilot_predictions,
            'ci_low':series_prepilot_predictions - rmse * sigma,
            'ci_up':series_prepilot_predictions + rmse * sigma,
            't':t_prepilot
        })
        df_prepilot_fact = pd.DataFrame({
            'data_type':'fact',
            'period_type':'prepilot',
            'values':series_prepilot_facts,
            'ci_low':series_prepilot_predictions - rmse * sigma,
            'ci_up':series_prepilot_predictions + rmse * sigma,
            't':t_prepilot
        })
        df_results = df_prepilot_fact.append(df_prepilot_pred).append(df_pilot_fact).append(df_pilot_pred)
        fig = px.line(
            df_results, 
            x="t", 
            y=['ci_low', "values", 'ci_up'], 
            color='data_type',
            color_discrete_sequence=["blue", "orange"]
        )
        fig.add_vline(
            x=min(t_pilot), 
            line_width=2, 
            line_dash="dash", 
            line_color="black", 
            opacity=0.7,
            annotation_text="A/A period start", 
            annotation_position="top left"
        )
        fig.show()

In [51]:
df_revenue_forecast = forecast_for_ab(df_prepilot, df_pilot, mtr = 'value')

R-squared: 0.9003, holdout 1.5 rmse (%): 0.07995
Control group size: 1, Test group size: 1
